# Multi-Instrumental Medley Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (Setup Environment)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools

!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import random
import copy
from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDIX

os.chdir('/content/')

In [ ]:
#@title Process MIDIs with TMIDIX MIDI Processor

full_path_to_MIDI_dataset_directory = "/content/Dataset/" #@param {type:"string"}
sorted_files_loading_order = False # @param {type:"boolean"}
dataset_ratio = 1 #@param {type:"slider", min:0.1, max:1, step:0.1}
pause_length_ms = 4000 # @param {type:"slider", min:0, max:8000, step:1000}
full_path_to_save_processed_MIDIs = "/content/Multi_Instrumental_Medley_Maker_Processed_MIDIs" #@param {type:"string"}

print('=' * 70)
print('TMIDIX MIDI Processor')
print('Starting up...')
print('=' * 70)

###########

files_count = 0

gfiles = []

melody_chords_f = []

###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = full_path_to_MIDI_dataset_directory

filez = list()

for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]

print('=' * 70)
print('Done!')
print('=' * 70)

if not filez:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:

  if sorted_files_loading_order:
      print('Sorting files...')
      filez.sort()
      print('=' * 70)
      print('Done!')
      print('=' * 70)
  else:
      print('Randomizing file list...')
      random.shuffle(filez)
      print('=' * 70)
      print('Done!')
      print('=' * 70)

  print('Processing MIDI files. Please wait...')
  print('=' * 70)

  for f in tqdm(filez[:int(len(filez) * dataset_ratio)]):
      try:
          fn = os.path.basename(f)
          fn1 = fn.split('.')[0]

          files_count += 1

          #print('Loading MIDI file...')
          score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read(), recalculate_channels=False)

          events_matrix1 = []

          itrack = 1

          while itrack < len(score):
              for event in score[itrack]:
                  events_matrix1.append(event)
              itrack += 1

          melody_chords_f.append(events_matrix1)

          gfiles.append(f)

      except KeyboardInterrupt:
          print('Saving current progress and quitting...')
          break

      except Exception as e:
          print('Error:', e)
          continue

  print('=' * 70)
  print('Saving...')
  print('=' * 70)
  TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, full_path_to_save_processed_MIDIs)
  print('=' * 70)
  print('Done!')
  print('=' * 70)

  print('Making Medley...')
  print('=' * 70)

  song = []

  time = 0

  for m in tqdm(melody_chords_f):

    pe = m[0]
    for mm in m:
      time += mm[1] - pe[1]

      mmm = copy.deepcopy(mm)
      mmm[1] = time
      song.append(mmm)
      pe = mm

    time += pause_length_ms

  print('=' * 70)
  print('Done!')
  print('=' * 70)

  print('Writing final MIDI...')

  detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song,
                                                          output_signature = 'Multi Instrumental Medley',
                                                          output_file_name = '/content/Multi-Instrumental-Medley',
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                                          )
  print('=' * 70)

# Congrats! You did it! :)